# import

In [40]:
import pandas as pd
pd.set_option('display.max_rows', 150)
pd.set_option('display.max_columns', 150)
import numpy as np
from utils import *
from itertools import combinations

In [64]:
PROJECT_NAME = "v2"

# load

In [87]:
fix_seed(42)
logger = init_logger()

In [39]:
with timer("load", logger):
    df_train = pd.read_csv("../input/train.csv")
    df_test = pd.read_csv("../input/test.csv")

df = pd.concat([df_train, df_test])

2022/01/02 13:05:58 46 [INFO] [load] start.
2022/01/02 13:06:04 48 [INFO] [load] done in 5.451 seconds.


# processing

In [41]:
drop_columns = [
    "Soil_Type7",
    "Soil_Type15"
]

df.drop(drop_columns, axis=1, inplace=True)

In [42]:
# aspect should be in [0, 360), meanwhile it is in (-360, 720)
df.loc[df["Aspect"] < 0, "Aspect"] += 360
df.loc[df["Aspect"] >= 360, "Aspect"] -= 360

In [43]:
df.head()

,Id,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area1,Wilderness_Area2,Wilderness_Area3,Wilderness_Area4,Soil_Type1,Soil_Type2,Soil_Type3,Soil_Type4,Soil_Type5,Soil_Type6,Soil_Type8,Soil_Type9,Soil_Type10,Soil_Type11,Soil_Type12,Soil_Type13,Soil_Type14,Soil_Type16,Soil_Type17,Soil_Type18,Soil_Type19,Soil_Type20,Soil_Type21,Soil_Type22,Soil_Type23,Soil_Type24,Soil_Type25,Soil_Type26,Soil_Type27,Soil_Type28,Soil_Type29,Soil_Type30,Soil_Type31,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
0,0,3189,40,8,30,13,3270,206,234,193,4873,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1.0
1,1,3026,182,5,280,29,3270,233,240,106,5423,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2.0
2,2,3106,13,7,351,37,2914,208,234,137,5269,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1.0
3,3,3022,276,13,192,16,3034,207,238,156,2866,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2.0
4,4,2906,186,13,266,22,2916,231,231,154,2642,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2.0


In [44]:
df["L1_Distance_To_Hydrology"] = np.abs(df["Horizontal_Distance_To_Hydrology"]) + np.abs(df["Vertical_Distance_To_Hydrology"])
df["L2_Distance_To_Hydrology"] = (df["Horizontal_Distance_To_Hydrology"]**2 + df["Vertical_Distance_To_Hydrology"]**2) ** .5
df.head()

,Id,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area1,Wilderness_Area2,Wilderness_Area3,Wilderness_Area4,Soil_Type1,Soil_Type2,Soil_Type3,Soil_Type4,Soil_Type5,Soil_Type6,Soil_Type8,Soil_Type9,Soil_Type10,Soil_Type11,Soil_Type12,Soil_Type13,Soil_Type14,Soil_Type16,Soil_Type17,Soil_Type18,Soil_Type19,Soil_Type20,Soil_Type21,Soil_Type22,Soil_Type23,Soil_Type24,Soil_Type25,Soil_Type26,Soil_Type27,Soil_Type28,Soil_Type29,Soil_Type30,Soil_Type31,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type,L1_Distance_To_Hydrology,L2_Distance_To_Hydrology
0,0,3189,40,8,30,13,3270,206,234,193,4873,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1.0,43,32.695565
1,1,3026,182,5,280,29,3270,233,240,106,5423,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2.0,309,281.497780
2,2,3106,13,7,351,37,2914,208,234,137,5269,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1.0,388,352.944755
3,3,3022,276,13,192,16,3034,207,238,156,2866,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2.0,208,192.665513
4,4,2906,186,13,266,22,2916,231,231,154,2642,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2.0,288,266.908224


In [45]:
wilderness_columns = [column for column in df if column.startswith("Wilderness")]
df["Count_Wilderness"] = df[wilderness_columns].sum(axis=1)

In [46]:
soil_columns = [column for column in df if column.startswith("Soil_Type")]
df["Count_Soil"] = df[soil_columns].sum(axis=1)

In [47]:
df.head()

,Id,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area1,Wilderness_Area2,Wilderness_Area3,Wilderness_Area4,Soil_Type1,Soil_Type2,Soil_Type3,Soil_Type4,Soil_Type5,Soil_Type6,Soil_Type8,Soil_Type9,Soil_Type10,Soil_Type11,Soil_Type12,Soil_Type13,Soil_Type14,Soil_Type16,Soil_Type17,Soil_Type18,Soil_Type19,Soil_Type20,Soil_Type21,Soil_Type22,Soil_Type23,Soil_Type24,Soil_Type25,Soil_Type26,Soil_Type27,Soil_Type28,Soil_Type29,Soil_Type30,Soil_Type31,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type,L1_Distance_To_Hydrology,L2_Distance_To_Hydrology,Count_Wilderness,Count_Soil
0,0,3189,40,8,30,13,3270,206,234,193,4873,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1.0,43,32.695565,1,1
1,1,3026,182,5,280,29,3270,233,240,106,5423,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2.0,309,281.497780,1,1
2,2,3106,13,7,351,37,2914,208,234,137,5269,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1.0,388,352.944755,1,1
3,3,3022,276,13,192,16,3034,207,238,156,2866,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2.0,208,192.665513,1,1
4,4,2906,186,13,266,22,2916,231,231,154,2642,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2.0,288,266.908224,1,1


In [93]:
# hillshade shoud be in [0, 255]

hillshade_columns = [column for column in df if column.startswith("Hillshade")]

for column in hillshade_columns:
    df[column] = df[column].clip(0, 255)

In [49]:
df["Sum_Hillshade"] = df[hillshade_columns].sum(axis=1)

In [50]:
for col0, col1 in combinations(hillshade_columns, 2):
    df[f"Diff_{col0}_{col1}"] = df[col0] - df[col1]

In [51]:
horizontal_columns = [*filter(lambda x: x.startswith("Horizontal"), df)]

for col0, col1 in combinations(horizontal_columns, 2):
    df[f"Diff_{col0}_{col1}"] = df[col0] - df[col1]

In [52]:
df_info(df)

,unique,missing,dtype,count,mean,std,min,25%,50%,75%,max
Id,5000000,0,int64,5000000.0,2.500000e+06,1.443376e+06,0.0,1.250000e+06,2.500000e+06,3.749999e+06,4.999999e+06
Elevation,2548,0,int64,5000000.0,2.978074e+03,3.005701e+02,1773.0,2.754000e+03,2.967000e+03,3.222000e+03,4.383000e+03
Aspect,360,0,int64,5000000.0,1.499568e+02,1.088817e+02,0.0,5.900000e+01,1.210000e+02,2.450000e+02,3.590000e+02
Slope,68,0,int64,5000000.0,1.506272e+01,8.529145e+00,-3.0,9.000000e+00,1.400000e+01,2.000000e+01,6.400000e+01
Horizontal_Distance_To_Hydrology,1645,0,int64,5000000.0,2.699260e+02,2.268518e+02,-136.0,1.090000e+02,2.110000e+02,3.600000e+02,1.602000e+03
Vertical_Distance_To_Hydrology,922,0,int64,5000000.0,5.162617e+01,6.827695e+01,-329.0,4.000000e+00,3.100000e+01,7.800000e+01,6.470000e+02
Horizontal_Distance_To_Roadways,7786,0,int64,5000000.0,1.756015e+03,1.327854e+03,-287.0,8.070000e+02,1.411000e+03,2.347000e+03,7.666000e+03
Hillshade_9am,256,0,int64,5000000.0,2.117821e+02,3.083571e+01,0.0,1.970000e+02,2.180000e+02,2.340000e+02,2.550000e+02
Hillshade_Noon,204,0,int64,5000000.0,2.211222e+02,2.214103e+01,49.0,2.100000e+02,2.240000e+02,2.370000e+02,2.550000e+02
Hillshade_3pm,256,0,int64,5000000.0,1.406678e+02,4.385597e+01,0.0,1.150000e+02,1.420000e+02,1.690000e+02,2.550000e+02


In [53]:
df_train, df_test = df[:len(df_train)], df[len(df_train):]

In [54]:
drop_index = df_train[df_train["Cover_Type"] == 5].index
drop_index

Int64Index([3403875], dtype='int64')

In [55]:
df_train.drop(drop_index, inplace=True)

/Users/g-ogaki/miniforge3/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [83]:
X_train = df_train.drop(["Id", "Cover_Type"], axis=1)
y_train = df_train["Cover_Type"].astype(np.int8)
X_test = df_test.drop(["Id", "Cover_Type"], axis=1)

y_train.value_counts()

2    2262087
1    1468136
3     195712
7      62261
6      11426
4        377
Name: Cover_Type, dtype: int64

In [59]:
from sklearn.preprocessing import LabelEncoder

In [84]:
le = LabelEncoder()
y_train = pd.Series(le.fit_transform(y_train))

# Training

In [73]:
from model_lgb import ModelLGB
from runner import Runner

In [74]:
params = {
    'bagging_fraction': 0.8691099275893169, 
    'feature_fraction': 0.9433525231687435, 
    'lambda_l1': 0.057908447715479115, 
    'lambda_l2': 0.01474021398017653, 
    'learning_rate': 0.01, 
    'max_leaves': 48, 
    'metric': ('multi_logloss', 'multi_error'), 
    'min_data_in_leaf': 26, 
    'objective': 'multiclass',
    "num_classes": 6,
    'seed': 0, 
    'verbose': -1
}

train_params = {
    "num_boost_round": 2000,
    "early_stopping_rounds": 20,
    "verbose_eval": 50,
}

In [75]:
runner = Runner(PROJECT_NAME, ModelLGB, cv=False)
runner.train(X_train, y_train, params, train_params)

/Users/g-ogaki/miniforge3/lib/python3.9/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/g-ogaki/miniforge3/lib/python3.9/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


Training until validation scores don't improve for 20 rounds
[50]	training's multi_logloss: 0.412508	training's multi_error: 0.0657247	valid_1's multi_logloss: 0.413344	valid_1's multi_error: 0.0658637
[100]	training's multi_logloss: 0.249584	training's multi_error: 0.0538956	valid_1's multi_logloss: 0.250759	valid_1's multi_error: 0.0541087
[150]	training's multi_logloss: 0.178138	training's multi_error: 0.0493028	valid_1's multi_logloss: 0.179556	valid_1's multi_error: 0.0497125
[200]	training's multi_logloss: 0.1434	training's multi_error: 0.046576	valid_1's multi_logloss: 0.145033	valid_1's multi_error: 0.0472575
[250]	training's multi_logloss: 0.123981	training's multi_error: 0.0444266	valid_1's multi_logloss: 0.125827	valid_1's multi_error: 0.0453925
[300]	training's multi_logloss: 0.111949	training's multi_error: 0.0425878	valid_1's multi_logloss: 0.113984	valid_1's multi_error: 0.0436088
[350]	training's multi_logloss: 0.104128	training's multi_error: 0.0412303	valid_1's multi_

In [76]:
runner.model.feature_importance_

Elevation                                                                   2.011613e+08
Count_Soil                                                                  1.057809e+07
Diff_Horizontal_Distance_To_Hydrology_Horizontal_Distance_To_Roadways       8.103541e+06
Horizontal_Distance_To_Roadways                                             7.867348e+06
Wilderness_Area3                                                            6.445394e+06
Diff_Horizontal_Distance_To_Hydrology_Horizontal_Distance_To_Fire_Points    5.388088e+06
Vertical_Distance_To_Hydrology                                              5.349658e+06
Wilderness_Area1                                                            4.701955e+06
Horizontal_Distance_To_Fire_Points                                          4.046873e+06
Diff_Horizontal_Distance_To_Roadways_Horizontal_Distance_To_Fire_Points     3.272390e+06
Wilderness_Area4                                                            2.151895e+06
Soil_Type39          

# inference

In [88]:
with timer("prediction", logger):
    prob = runner.predict(X_test)

prob

2022/01/02 13:32:02 46 [INFO] [prediction] start.
2022/01/02 13:32:02 46 [INFO] [prediction] start.
2022/01/02 13:33:16 48 [INFO] [prediction] done in 73.641 seconds.
2022/01/02 13:33:16 48 [INFO] [prediction] done in 73.641 seconds.


array([[3.36619157e-05, 9.99835530e-01, 1.28587610e-04, 4.89611023e-09,
        1.50325444e-06, 7.12450095e-07],
       [1.37330565e-02, 9.86153922e-01, 7.67985847e-05, 3.85585105e-08,
        2.93994590e-05, 6.78449268e-06],
       [2.85309869e-03, 9.97131825e-01, 1.29646858e-05, 1.00109336e-08,
        3.50051164e-07, 1.75168919e-06],
       ...,
       [1.10528499e-04, 9.99771648e-01, 1.13202218e-04, 1.18115499e-08,
        2.28970965e-06, 2.31973603e-06],
       [9.99341469e-01, 4.27270142e-04, 5.15522312e-06, 2.07411706e-08,
        6.16899976e-07, 2.25468020e-04],
       [1.93399716e-04, 1.48365812e-01, 8.51314837e-01, 4.55391854e-08,
        1.20103592e-04, 5.80226378e-06]])

In [89]:
pred = prob.argmax(axis=1)
pred = le.inverse_transform(pred)
pred

array([2, 2, 2, ..., 2, 1, 3], dtype=int8)

In [90]:
res = pd.DataFrame({
    "Id": df_test["Id"],
    "Cover_Type": pred
})
res.to_csv(f"../submission/submission_{PROJECT_NAME}.csv", index=False)
res

,Id,Cover_Type
0,4000000,2
1,4000001,2
2,4000002,2
3,4000003,2
4,4000004,2
...,...,...
999995,4999995,2
999996,4999996,1
999997,4999997,2
999998,4999998,1


In [92]:
!kaggle competitions submit tabular-playground-series-dec-2021 -f ../submission/submission_{PROJECT_NAME}.csv -m "valid_1's multi_error: 0.03877"

100%|███████████████████████████████████████| 9.54M/9.54M [00:15<00:00, 660kB/s]
Successfully submitted to Tabular Playground Series - Dec 2021